## RAG_Clova

#### Env

Python Version : Python 3.12.2

requirements : 
https://ssl.pstatic.net/static/clova/service/hyperclova/cookbook/rag/requirements.txt

In [3]:
# Vector DB인 Milvus와 관련된 모듈들은 모듈의 용도를 명확히 구분하기 위해 Vector DB 구축 단계에서 불러왔으며 해당 부분에서 코드를 확인하실 수 있습니다.
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests
import dotenv
dotenv.load_dotenv()
# CLOVA_api_key=os.getenv("X-NCP-CLOVASTUDIO-API-KEY")
# NCP_API_api_key=os.getenv("X-NCP-APIGW-API-KEY")
# clova_REQUEST_ID=os.getenv("X-NCP-CLOVASTUDIO-REQUEST-ID")

True

#### 1. Raw Data → Connecting

In [4]:
url_to_filename_map = {}
wget_path="/opt/homebrew/bin/wget"
with open("langchainwikiurl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]
 
folder_path = "langchainwikiguide"
 
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    subprocess.run([wget_path, "-O", file_path, url], check=True)
    url_to_filename_map[url] = filename
 
with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)

--2024-09-16 02:20:14--  https://teddylee777.github.io/poetry/poetry-tutorial/
Resolving teddylee777.github.io (teddylee777.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to teddylee777.github.io (teddylee777.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44351 (43K) [text/html]
Saving to: ‘langchainwikiguide/.html’

     0K .......... .......... .......... .......... ...       100% 6.82M=0.006s

2024-09-16 02:20:14 (6.82 MB/s) - ‘langchainwikiguide/.html’ saved [44351/44351]

--2024-09-16 02:20:14--  https://teddylee777.github.io/langchain/langchain-agent/
Resolving teddylee777.github.io (teddylee777.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to teddylee777.github.io (teddylee777.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139959 (137K) [text/html]
Saving to: ‘langchainwikiguide/.html’

     0K .......... .

##### 보안 에러 나는 경우 방법

In [19]:
import os
import json
import requests
from tqdm import tqdm

url_to_filename_map = {}
folder_path = "langchainwikiguide"

with open("langchainwikiurl.txt", "r",encoding='utf8') as file:
    urls = [url.strip() for url in file.readlines()]
print(urls)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

for url in tqdm(urls, desc="Downloading"):
    filename = url.split("/")[-3] + ".html"
    file_path = os.path.join(folder_path, filename)
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        
        url_to_filename_map[url] = filename

    except requests.RequestException as e:
        print(f"An error occurred while downloading {url}: {e}")

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file, indent=4)

print("Download complete. Mapping saved to url_to_filename_map.json")


['https://teddylee777.github.io/poetry/poetry-tutorial/', 'https://teddylee777.github.io/langchain/langchain-agent/', 'https://teddylee777.github.io/langchain/summarize-chain/', 'https://teddylee777.github.io/langchain/rag-naver-news-qa/', 'https://teddylee777.github.io/openai/openai-models/']


Downloading: 100%|██████████| 5/5 [00:00<00:00, 37.96it/s]

Download complete. Mapping saved to url_to_filename_map.json


##### Quote

txt 파일을 작성할 때는 한 줄에 하나의 URL을 작성해야 합니다. 이때, 각 URL은 줄바꿈으로 구분되어야 합니다.

그리고 html 데이터를 로딩하기 전, robots.txt를 통해 데이터로 활용할 사이트의 접근 허용 여부를 확인해야합니다. NCP의 클로바스튜디오 사용 가이드의 경우 User-agent: * Allow:/ 로 접근이 가능합니다.

예제에서 활용한 HTML은, 네이버클라우드 플랫폼(NCP)의 클로바스튜디오 사용 가이드중 CLOVA Studio("AI Services" → "CLOVA Studio")와 관련된 모든 안내 페이지를 데이터로 활용했습니다.

LangChain을 활용해 로딩한 html은, 파일을 저장한 디렉토리의 주소를 metadata의 'source'로 가져오게 됩니다. 추후 답변 제공시, 디렉토리 주소가 아닌 실제 URL을 제공하기 위해, LangChain이 로딩한 데이터를 수정해야합니다.

html을 로딩할 때 파일명을 URL로 할 경우, /와 : 기호로 인해 파일명이 깨지게 됩니다. 따라서 원본 URL과 로딩한 HTML 파일을 쌍으로 mapping하고, 이 정보를 json 형식으로 "url_to_filename_map"에 저장합니다.

이후, html 파일의 저장 경로가 담긴 'source'를, 이 json 파일과 연결해 실제 URL로 변환해줍니다. 위 코드는, json 파일을 저장하는 단계까지입니다.

##### LangChain 활용 HTML 로딩



In [6]:
# 폴더 이름에 맞게 수정
html_files_dir = Path('langchainwikiguide')
 
html_files = list(html_files_dir.glob("*.html"))
 
langchainwikidatas = []
 
for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    langchainwikidatas.append(document_data)
    print(f"Processed {html_file}")

Processed langchainwikiguide/.html
Processed langchainwikiguide/233341.html
Processed langchainwikiguide/233346.html
Processed langchainwikiguide/233344.html
Processed langchainwikiguide/233345.html
Processed langchainwikiguide/250954.html
Processed langchainwikiguide/233342.html
Processed langchainwikiguide/233343.html


##### Quote

각 HTML에는 page_content에 모든 텍스트가, metadata의 'source'에는 저장 경로가 기록되어 있습니다. 위는, 로딩된 데이터 중 하나인 clovastudio-info.html의 형태입니다. metadata의 'source'에 실제 URL이 아닌 html 파일이 저장된 디렉토리 경로가 담긴 것을 확인할 수 있습니다. Mapping을 통해 이 'source'를 실제 URL로 바꾸는 작업을 다음 단계에 진행하게 됩니다.

##### Mapping 정보를 활용해 'source'를 실제 URL로 대체

In [7]:
with open("url_to_filename_map.json", "r") as map_file:
    url_to_filename_map = json.load(map_file)
 
filename_to_url_map = {v: k for k, v in url_to_filename_map.items()}
 
# langchainwikidatas 리스트의 각 Document 객체의 'source' 수정
for doc_list in langchainwikidatas:
    for doc in doc_list:
        extracted_filename = doc.metadata["source"].split("/")[-1]
        if extracted_filename in filename_to_url_map:
            doc.metadata["source"] = filename_to_url_map[extracted_filename]
        else:
            print(f"Warning: {extracted_filename}에 해당하는 URL을 찾을 수 없습니다.")

In [8]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
langchainwiki_flattened = [item for sublist in langchainwikidatas for item in sublist]

In [9]:
langchainwiki_flattened 

[Document(page_content='🔥알림🔥 ① 테디노트 유튜브 - 구경하러 가기! ② LangChain 한국어 튜토리얼 바로가기 👀 ③ 랭체인 노트 무료 전자책(wikidocs) 바로가기 🙌 ④ RAG 비법노트 LangChain 강의오픈 바로가기 🙌 ⑤ 서울대 PyTorch 딥러닝 강의 바로가기 🙌\n\nOpenAI API 모델 리스트 / 요금표\n\n2024년 02월 02일 2 분 소요\n\n최신 버전의 업데이트 된 OpenAI Model 리스트와 API 사용요금(Pricing) 입니다.\n\n상세한 업데이트 내역은 여기 에서 확인하실 수 있습니다.\n\n업데이트 일자: 2024.01.25\n\n모델 리스트 및 요금표\n\n[참고]\n\n개발시 모델 변경에 효과적으로 대처하기 위하여 다음의 model alias 를 적용 및 업데이트 하였습니다.\n\n적용은 모델 업데이트 후 2주 후 에 적용예정입니다.\n\n(2024.02.08 이후부터 적용)\n\ngpt-4-turbo-preview ➡️ gpt-4-0125-preview\n\ngpt-3.5-turbo ➡️ gpt-3.5-turbo-0125\n\nGPT-4 Turbo & GPT-4 & GPT 3.5 Turbo\n\nModel Name Input Cost Output Cost GPT-4 Turbo gpt-4-0125-preview $0.01 / 1K tokens $0.03 / 1K tokens gpt-4-1106-preview $0.01 / 1K tokens $0.03 / 1K tokens gpt-4-1106-vision-preview $0.01 / 1K tokens $0.03 / 1K tokens GPT-4 gpt-4 $0.03 / 1K tokens $0.06 / 1K tokens gpt-4-32k $0.06 / 1K tokens $0.12 / 1K tokens GPT 3.5 Turbo gpt-3.5-turbo-0125 $0.0005 / 1K tokens $0.0015 / 1K tokens gpt-

#### 2. Chunking



임베딩 모델이 처리할 수 있는 적당한 크기로 raw data를 나누는 것은 매우 중요합니다. 이는 임베딩 모델마다 한 번에 처리할 수 있는 토큰 수의 한계가 있기 때문입니다. CLOVA Studio의 문단 나누기 API는 모델이 직접 문장들간의 의미 유사도를 찾아 최적의 chunk 개수와 사용자가 원하는 1개 chunk의 크기(글자 수)를 직접 설정하여 문단을 나눌 수도 있습니다. 추가로, 후처리(postProcess = True)를 통해 chunk당 글자 수의 상한선과 하한선을 postProcessMaxSize와 postProcessMinSize로 조절할 수도 있습니다.

In [10]:
class SegmentationExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }
 
        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            "POST",
            "/testapp/v1/api-tools/segmentation/90972c6f0a784c9886f4bc6ae8a66c01", # If using Service App, change 'testapp' to 'serviceapp', and corresponding app id.
            json.dumps(completion_request),
            headers
        )
        response = conn.getresponse()
        print(response.length)
        result = json.loads(response.read().decode(encoding="utf-8"))
        print(result)
        conn.close()
        return result
 
    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["topicSeg"]
        else:
            raise ValueError(f"{res}")
 
 
if __name__ == "__main__":
    segmentation_executor = SegmentationExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key="NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/",
        api_key_primary_val="J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB",
        request_id=""
    )
 
    chunked_html = []
 
    for htmldata in tqdm(langchainwiki_flattened ):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": 1.5,
                "segCnt": -1,
                "postProcessMinSize": 0,
                "text": htmldata.page_content,
                "postProcess": False
            }

            request_json_string = json.dumps(request_data)
            request_data = json.loads(request_json_string, strict=False)
            print(1)
            response_data = segmentation_executor.execute(request_data)
        except Exception as e:
            print(f"Error occurred. Message: {e}")
        
        for paragraph in response_data:
            chunked_document = {
                "source": htmldata.metadata["source"],
                "text": paragraph
            }
            chunked_html.append(chunked_document)
 
print(len(chunked_html))

  0%|          | 0/8 [00:00<?, ?it/s]

1


 12%|█▎        | 1/8 [00:01<00:09,  1.35s/it]

6631
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['🔥알림🔥 ① 테디노트 유튜브 - 구경하러 가기!'], ['② LangChain 한국어 튜토리얼 바로가기 👀 ③ 랭체인 노트 무료 전자책(wikidocs) 바로가기 🙌 ④ RAG 비법노트 LangChain 강의오픈 바로가기 🙌 ⑤ 서울대 PyTorch 딥러닝 강의 바로가기 🙌'], ['OpenAI API 모델 리스트 / 요금표'], ['2024년 02월 02일 2 분 소요'], ['최신 버전의 업데이트 된 OpenAI Model 리스트와 API 사용요금(Pricing) 입니다.', '상세한 업데이트 내역은 여기 에서 확인하실 수 있습니다.'], ['업데이트 일자: 2024.01.25'], ['모델 리스트 및 요금표'], ['[참고]'], ['개발시 모델 변경에 효과적으로 대처하기 위하여 다음의 model alias 를 적용 및 업데이트 하였습니다.', '적용은 모델 업데이트 후 2주 후 에 적용예정입니다.'], ['(2024.02.08 이후부터 적용)'], ['gpt-4-turbo-preview ➡️ gpt-4-0125-preview', 'gpt-3.5-turbo ➡️ gpt-3.5-turbo-0125'], ['GPT-4 Turbo & GPT-4 & GPT 3.5 Turbo'], ['Model Name Input Cost Output Cost GPT-4 Turbo gpt-4-0125-preview $0.01 / 1K tokens $0.03 / 1K tokens gpt-4-1106-preview $0.01 / 1K tokens $0.03 / 1K tokens gpt-4-1106-vision-preview $0.01 / 1K tokens $0.03 / 1K tokens GPT-4 gpt-4 $0.03 / 1K tokens $0.06 / 1K tokens gpt-4-32k $0.06 / 1K tokens $0.12 / 1K tok

 25%|██▌       | 2/8 [00:03<00:12,  2.01s/it]

...

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. 설치 영상보고 따라하기'], ['02. OpenAI API 키 발급 및 테스트'], ['03. LangSmith 추적 설정'], ['04. OpenAI API 사용(GPT-4o 멀티모달)'], ['05. LangChain Expression Language(LCEL)'], ['06. LCEL 인터페이스'], ['07. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative 

 50%|█████     | 4/8 [00:12<00:13,  3.45s/it]

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. 설치 영상보고 따라하기'], ['02. OpenAI API 키 발급 및 테스트'], ['03. LangSmith 추적 설정'], ['04. OpenAI API 사용(GPT-4o 멀티모달)'], ['05. LangChain Expression Language(LCEL)'], ['06. LCEL 인터페이스'], ['07. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative 

 50%|█████     | 4/8 [00:14<00:14,  3.50s/it]


KeyboardInterrupt: 

In [ ]:
len(chunked_html)

1507

In [ ]:
chunked_html

[{'source': 'https://wikidocs.net/233341',
  'text': ['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기']},
 {'source': 'https://wikidocs.net/233341', 'text': ['01. 설치 영상보고 따라하기']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['02. OpenAI API 키 발급 및 테스트']},
 {'source': 'https://wikidocs.net/233341', 'text': ['03. LangSmith 추적 설정']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['04. OpenAI API 사용(GPT-4o 멀티모달)']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['05. LangChain Expression Language(LCEL)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['06. LCEL 인터페이스']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['07. Runnable CH02 프롬프트(Prompt)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['01. 프롬프트(Prompt)']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['02. 퓨샷 프롬프트(FewShotPromptTemplate)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['03. LangChain Hub']},
 {'source': 'https://wikidocs.net/233341',
  'text

#### 3. Embedding

문단 나누기 API를 통해 나누어진 525개의 chunk(chunked_html)을 CLOVA Studio의 임베딩 API를 사용해 1024차원의 벡터로 변환하는 과정입니다. clir-emb-dolphin의 경우, 임베딩 과정에서 유사도 판단을 위해 벡터의 내적(Inner Product, IP)을 거리 단위로 사용합니다. 반면, clir-sts-dolphin은 코사인 거리(Cosine)를 거리 단위로 사용합니다. 이후, 벡터의 인덱싱과 검색 과정에서 사용자는 어떤 거리 단위를 사용하여 데이터와 사용자의 쿼리 간 유사도를 판단할 것인지 선택할 수 있습니다.

거리 단위를 임베딩부터 인덱싱, 검색까지 일치시켜야 데이터의 품질이 향상되므로, 임베딩 과정에서 사용한 모델(emb / sts)을 기억하는 것이 중요합니다. 임베딩 클래스에 오류가 발생시 멈추게끔 하는 로직을 일부 추가해, 다량의 데이터를 처리할 때 오류 발생시 재실행의 부담을 줄입니다.

In [ ]:
class EmbeddingExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            "POST",
            "/testapp/v1/api-tools/embedding/v2/95709fb7cc5047b6adad8f4d8442bf24",
            json.dumps(completion_request),
            headers
        )
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding="utf-8"))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["embedding"]
        else:
            error_code = res["status"]["code"]
            error_message = res.get("status", {}).get("message", "Unknown error")
            raise ValueError(f"오류 발생: {error_code}: {error_message}")

if __name__ == "__main__":
    embedding_executor = EmbeddingExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key='NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/',
        api_key_primary_val='J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB',
        request_id='1037451f-6a88-40ae-afd4-fbc4a26cb354'
    )

    for i, chunked_document in enumerate(tqdm(chunked_html)):
        if i==100:
            break
        try:
            text_content = chunked_document['text']
            if isinstance(text_content, list):
                text_content=text_content[0]
            request_json = {
                "text": text_content
            }
            response_data = embedding_executor.execute(request_json)
            chunked_document["embedding"] = response_data
        except ValueError as e:
            print(f"Embedding API Error. {e}")
            print(chunked_document['text'])
        except Exception as e:
            print(f"Unexpected error: {e}")
        


  4%|▎         | 54/1507 [00:13<02:13, 10.92it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['13. UpstageLayoutAnalysisLoader']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['14. LlamaParser CH07 텍스트 분할(Text Splitter)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['01. 문자 텍스트 분할(CharacterTextSplitter)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['02. 재귀적 문자 텍스트 분할(RecursiveCharacterTextSplitter)']


  4%|▎         | 56/1507 [00:13<02:10, 11.16it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['03. 토큰 텍스트 분할(TokenTextSplitter)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['04. 시멘틱 청커(SemanticChunker)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['05. 코드 분할(Python, Markdown, JAVA, C++, C#, GO, JS, Latex 등)']


  4%|▍         | 60/1507 [00:13<01:48, 13.32it/s]

...

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['05. 대화내용을 기억하는 RAG 체인 CH13 LangChain Expression Language(LCEL)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['01. RunnablePassthrough']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['02. Runnable 구조(그래프) 검토']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['03. RunnableLambda']


  6%|▋         | 96/1507 [00:16<01:27, 16.12it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['04. LLM 체인 라우팅(RunnableLambda, RunnableBranch)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['05. RunnableParallel']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['06. 동적 속성 지정(configurable_fields, configurable_alternatives) 07. @chain 데코레이터로 Runnable 구성']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['08. RunnableWithMessageHistory']


  7%|▋         | 100/1507 [00:16<03:50,  6.10it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['09. 사용자 정의 제네레이터(generator)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['10. Runtime Arguments 바인딩']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['11. 폴백(fallback) 모델 지정 CH14 체인(Chains)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['01. 문서 요약']


In [ ]:
chunked_document

{'source': 'https://wikidocs.net/233341', 'text': ['02. SQL']}

In [ ]:
dimension_set = set()
 
for item in chunked_html:
    if "embedding" in item:
        dimension = len(item["embedding"])
        dimension_set.add(dimension)
 
print("임베딩된 벡터들의 차원:", dimension_set)

임베딩된 벡터들의 차원: {1024}


In [ ]:
chunked_html[1]

{'source': 'https://wikidocs.net/233341',
 'text': ['01. 설치 영상보고 따라하기'],
 'embedding': [-0.14343262,
  -0.051239014,
  -1.1572266,
  1.1259766,
  -0.37841797,
  0.23815918,
  0.43945312,
  0.38452148,
  0.18981934,
  0.25830078,
  0.8911133,
  -0.8701172,
  0.16271973,
  0.76708984,
  -0.7871094,
  -0.6538086,
  -0.13928223,
  -0.4814453,
  1.3896484,
  -0.2746582,
  0.4411621,
  -0.7993164,
  1.6367188,
  0.7055664,
  0.48461914,
  -0.09399414,
  -0.58447266,
  -0.8251953,
  1.6621094,
  -0.4465332,
  -0.9316406,
  -0.7988281,
  1.0195312,
  -0.55029297,
  -0.70458984,
  -1.3769531,
  0.2298584,
  -0.59277344,
  -0.6894531,
  1.1083984,
  0.08477783,
  0.10827637,
  -0.009757996,
  -0.42871094,
  -0.10406494,
  -0.73779297,
  -1.6005859,
  -0.23571777,
  -0.8544922,
  0.1171875,
  0.18493652,
  -0.06384277,
  0.014839172,
  -0.7495117,
  -0.14355469,
  1.3154297,
  -0.66796875,
  -0.85595703,
  -1.6630859,
  0.04333496,
  -0.7998047,
  1.0253906,
  -1.2841797,
  -0.50097656,
  0.39086

#### 4. Vector DB

CLOVA Studio의 임베딩 API를 활용하여 모든 chunked_의 text를 1024차원의 벡터로 변환한 후, 'embedding'이라는 이름의 객체로 chunked_html에 추가해 주었습니다. 이제 이 데이터를 Vector DB에 저장하고, 인덱싱을 하는 단계입니다. Vector DB로는 Milvus를 사용하였으며, 대시보드는 Docker를 사용하여 DB를 켜고(Run) 끄는(Stop) 작업을 수행하였습니다.

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

In [ ]:
connections.connect()
  
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=3000),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=9000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024)
]

schema = CollectionSchema(fields, description="랭체인 한국어 튜토리얼 가이드 입니다.")

collection_name = "langchainwiki"
collection = Collection(name=collection_name, schema=schema, using='default', shards_num=2)

for item in chunked_html:
    try:
        source = item['source']
        text = item['text'][0] if isinstance(item['text'], list) else item['text']  # 리스트인 경우 첫 번째 요소를 사용
        embedding = item.get('embedding')  # get() 메소드를 사용하여 'embedding' 키가 없을 경우 None을 반환
        
        if embedding is None:
            print(f"경고: 'embedding' 키가 없는 항목을 건너뜁니다. 항목: {item}")
            continue  # 'embedding'이 없는 항목은 건너뜁니다
        
        entities = [
            [source],  # source는 리스트로 감싸줍니다
            [text],    # text도 리스트로 감싸줍니다
            [embedding]
        ]
        
        insert_result = collection.insert(entities)
        print("데이터 Insertion이 완료된 ID:", insert_result.primary_keys)
    except Exception as e:
        print(f"데이터 Insertion 중 오류 발생: {e}")
        print(f"문제가 발생한 데이터: {item}")

print("데이터 Insertion이 전부 완료되었습니다")

데이터 Insertion이 완료된 ID: [452570396700182048]
데이터 Insertion이 완료된 ID: [452570396700182050]
데이터 Insertion이 완료된 ID: [452570396700182052]
데이터 Insertion이 완료된 ID: [452570396700182054]
데이터 Insertion이 완료된 ID: [452570396700182056]
데이터 Insertion이 완료된 ID: [452570396700182058]
데이터 Insertion이 완료된 ID: [452570396700182060]
데이터 Insertion이 완료된 ID: [452570396700182062]
데이터 Insertion이 완료된 ID: [452570396700182064]
데이터 Insertion이 완료된 ID: [452570396700182066]
데이터 Insertion이 완료된 ID: [452570396700182068]
데이터 Insertion이 완료된 ID: [452570396700182070]
데이터 Insertion이 완료된 ID: [452570396700182072]
데이터 Insertion이 완료된 ID: [452570396700182074]
데이터 Insertion이 완료된 ID: [452570396700182076]
데이터 Insertion이 완료된 ID: [452570396700182078]
데이터 Insertion이 완료된 ID: [452570396700182080]
데이터 Insertion이 완료된 ID: [452570396700182082]
데이터 Insertion이 완료된 ID: [452570396700182084]
데이터 Insertion이 완료된 ID: [452570396700182086]
데이터 Insertion이 완료된 ID: [452570396700182088]
데이터 Insertion이 완료된 ID: [452570396700182090]
데이터 Insertion이 완료된 ID: [45257039

활용 SDK가 업데이트 될 경우 코드에 변화가 발생할 수 있습니다.
데이터 chunk가 에러가 발생하여 임베딩 값을 가지고 있지 않을 경우 오류가 발생할 수 있으니, 임베딩이 정상적으로 완료 되었는지 확인해야 합니다.

#### Indexing

앞서 임베딩 시 거리 단위를 IP로 설정했기 때문에, 인덱싱에서도 "metric_type"을 "IP"로 설정합니다. 인덱싱 유형은 다양한 알고리즘이 존재하며, 최근에는 HNSW 방식이 많이 활용됩니다. 하지만 선택한 Vector DB가 어떤 인덱싱 유형을 지원하는지 확인해야 합니다. "M"은 각 노드(데이터 포인트)가 유지할 수 있는 최대 엣지(연결)의 수로, 값이 높을수록 검색 정확도는 높아지지만 인덱스 생성 시간과 메모리 사용량도 증가하는 trade-off 관계를 가지고 있습니다.

"efConstruction"은 인덱스 구축 시 만들 그래프 구조의 깊이와 너비로, 값이 높을수록 정확도는 올라가지만 인덱스 생성 시간과 메모리 사용량도 증가하는 trade-off 관계를 가지고 있습니다. "M" : 8, "efConstruction" : 200의 설정은 Milvus가 공개한 Milvus 2.2.0의 성능 벤치마크에서 실험 시 설정한 파라미터 값을 가져온 것으로, Cookbook도 Milvus 2.2.x 환경에서 진행되었습니다. 데이터의 양이 많을수록 인덱싱에 시간이 오래 소요될 수 있습니다. 데이터가 많은 경우, 인덱싱 생성 후 바로 결과를 확인하면 결과를 확인할 수 없으며, 인덱싱은 하나의 객체와 값으로 저장되는 것이 아니기 때문에 아래와 같은 실행 결과로 인덱싱이 완료되었는지 확인할 수 있습니다.

In [ ]:
index_params = {
    "metric_type": "IP",
    "index_type": "HNSW",
    "params": {
        "M": 8,
        "efConstruction": 200
    }
}
 
collection = Collection("langchainwiki")
collection.create_index(field_name="embedding", index_params=index_params)
utility.index_building_progress("langchainwiki")
print([index.params for index in collection.indexes])

[{'metric_type': 'IP', 'index_type': 'HNSW', 'params': {'M': 8, 'efConstruction': 200}}]


In [ ]:
connections.connect("default", host="localhost", port="19530")
 
# 불러올 collection 이름을 넣는 곳
collection = Collection("langchainwiki")
utility.load_state("langchainwiki")

<LoadState: NotLoad>

##### 1. 가장 긴 길이의 답변을 선택하는 경우

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def execute(self, completion_request, response_type="stream"):
        headers = {
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id,
            "Content-Type": "application/json; charset=utf-8",
            "Accept": "text/event-stream"
        }
 
        final_answer = ""
 
        with requests.post(
            self._host + "/testapp/v1/chat-completions/HCX-003",
            headers=headers,
            json=completion_request,
            stream=True
        ) as r:
            if response_type == "stream":
                longest_line = ""
                for line in r.iter_lines():
                    if line:
                        decoded_line = line.decode("utf-8")
                        if decoded_line.startswith("data:"):
                            event_data = json.loads(decoded_line[len("data:"):])
                            message_content = event_data.get("message", {}).get("content", "")
                            if len(message_content) > len(longest_line):
                                longest_line = message_content
                final_answer = longest_line
            elif response_type == "single":
                final_answer = r.json()  # 가정: 단일 응답이 JSON 형태로 반환됨

##### 2."[DONE]" 바로 직전의 출력 결과를 선택하는 경우

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def execute(self, completion_request, response_type="stream"):
        headers = {
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id,
            "Content-Type": "application/json; charset=utf-8",
            "Accept": "text/event-stream"
        }
 
        final_answer = ""
 
        # URL 수정
        url = f"{self._host}/testapp/v1/chat-completions/HCX-DASH-001"
 
        with requests.post(
            url,
            headers=headers,
            json=completion_request,
            stream=True
        ) as r:
            if response_type == "stream":
                longest_line = ""
                for line in r.iter_lines():
                    if line:
                        decoded_line = line.decode("utf-8")
                        if decoded_line.startswith("data:"):
                            event_data = json.loads(decoded_line[len("data:"):])
                            message_content = event_data.get("message", {}).get("content", "")
                            if len(message_content) > len(longest_line):
                                longest_line = message_content
                final_answer = longest_line
            elif response_type == "single":
                final_answer = r.json()  # 가정: 단일 응답이 JSON 형태로 반환됨
        
        return final_answer

Stream 형태의 답변 구조에서는 다음과 같은 두 가지 로직을 사용하여 최종 full-length 답변을 추출할 수 있습니다.

최종 full-length 답변은 출력 결과들 중 가장 긴 답변이므로, 모든 출력 결과의 길이를 비교하여 가장 긴 답변을 선택합니다.
Stream 답변 구조상 "[DONE]" 표시 직전의 출력 결과가 최종 full-length 답변이므로, "[DONE]" 표시 이전의 마지막 출력 결과를 선택합니다.
Cookbook에서는 1번 로직을 사용하여 클래스를 정의했습니다.

In [ ]:
# 사용자의 쿼리를 임베딩하는 함수를 먼저 정의
def query_embed(text: str):
    request_data = {"text": text}
    response_data = embedding_executor.execute(request_data)
    return response_data

In [ ]:
def html_chat(realquery: str) -> str:
    # 사용자 쿼리 벡터화
    query_vector = query_embed(realquery)
 
    collection.load()
 
    search_params = {"metric_type": "IP", "params": {"ef": 64}}
    results = collection.search(
        data=[query_vector],  # 검색할 벡터 데이터
        anns_field="embedding",  # 검색을 수행할 벡터 필드 지정
        param=search_params,
        limit=10,
        output_fields=["source", "text"]
    )
 
    reference = []
 
    for hit in results[0]:
        distance = hit.distance
        source = hit.entity.get("source")
        text = hit.entity.get("text")
        reference.append({"distance": distance, "source": source, "text": text})
 
    completion_executor = CompletionExecutor(
        host="https://clovastudio.stream.ntruss.com",
        api_key='NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/',
        api_key_primary_val='J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB',
        request_id='eec0805e-5ae2-4d9c-872a-b6830bf55d2c'
    )
 
    preset_texts = [
        {
            "role": "system",
            "content": "- 너의 역할은 사용자의 질문에 reference를 바탕으로 답변하는거야. \n- 너가 가지고있는 지식은 모두 배제하고, 주어진 reference의 내용만을 바탕으로 답변해야해. \n- 답변의 출처가 되는 html의 내용인 'source'도 답변과 함께 {url:}의 형태로 제공해야해. \n- 만약 사용자의 질문이 reference와 관련이 없다면, {제가 가지고 있는 정보로는 답변할 수 없습니다.}라고만 반드시 말해야해."
        }
    ]
 
    for ref in reference:
        preset_texts.append(
            {
                "role": "system",
                "content": f"reference: {ref['text']}, url: {ref['source']}"
            }
        )
 
    preset_texts.append({"role": "user", "content": realquery})
 
    request_data = {
        "messages": preset_texts,
        "topP": 0.6,
        "topK": 0,
        "maxTokens": 1024,
        "temperature": 0.5,
        "repeatPenalty": 1.2,
        "stopBefore": [],
        "includeAiFilters": False
    }
 
    # LLM 생성 답변 반환
    response_data = completion_executor.execute(request_data)
 
    return response_data

In [ ]:
response=html_chat("langchain의 주요 기능이 뭐야?")
print(response)

랭체인(LangChain)의 주요 기능은 다음과 같습니다.  

1. 자동 번역: 자동 번역 기능을 제공합니다. 사용자가 입력한 언어를 다른 언어로 자동으로 번역해줍니다.

2. 다국어 지원: 다국어 지원 기능을 제공합니다. 여러 가지 언어를 지원하며, 사용자가 원하는 언어를 선택하여 사용할 수 있습니다.

3. 번역 관리: 번역 관리 기능을 제공합니다. 번역 내역을 관리할 수 있으며, 번역 품질을 평가하고 개선할 수 있습니다.

4. 번역 커뮤니티: 번역 커뮤니티 기능을 제공합니다. 사용자들이 서로 번역을 공유하고, 의견을 나눌 수 있는 커뮤니티를 제공합니다.

5. 번역 도구: 번역 도구 기능을 제공합니다. 번역에 필요한 다양한 도구를 제공하며, 사용자가 편리하게 번역을 할 수 있도록 도와줍니다.

6. 자동 문장 생성: 자동 문장 생성 기능을 제공합니다. 사용자가 입력한 단어를 기반으로 자동으로 문장을 생성해줍니다.

7. 언어 학습: 언어 학습 기능을 제공합니다. 사용자가 언어를 학습할 수 있도록 다양한 학습 자료를 제공합니다.

8. 번역 AI: 번역 AI 기능을 제공합니다. 인공지능 기술을 활용하여 번역을 자동으로 수행해주는 기능입니다.

9. 다국어 채팅: 다국어 채팅 기능을 제공합니다. 여러 가지 언어로 채팅을 할 수 있는 기능입니다.

10. 다국어 이메일: 다국어 이메일 기능을 제공합니다. 여러 가지 언어로 이메일을 작성할 수 있는 기능입니다.


In [ ]:
response=html_chat("LangSmith의 주요 기능이 뭐야?")
print(response)

LangSmith는 다음과 같은 주요 기능을 제공합니다.

1. 코드 자동 완성: 자동완성 기능을 제공하여 코드 작성을 쉽고 빠르게 할 수 있습니다.
2. 코드 분석: 코드의 오류를 감지하고, 코드의 성능을 개선할 수 있는 방법을 제안합니다.
3. 코드 생성: 새로운 코드를 자동으로 생성해줍니다.  

자세한 내용은 LangSmith 공식 문서를 참고해주세요.

- LangSmith 공식 문서: https://********.***/233341
